# Test problem from Caroe et al. (1999)

$$
\begin{align*}
\max_{x}      \quad & \frac{3}{2} x_1 + 4 x_2 + Q(x_1, x_2) \\
\mathrm{s.t.} \quad & 0 \leq x \leq 5 \\
                    & x \ \mathrm{integer}
\end{align*}
$$

where $Q(x_1,x_2)$:
$$
\begin{align*}
\max_{y}      \quad & 16 y_1 + 19 y_2 + 23 y_3 + 28 y_4 \\
\mathrm{s.t.} \quad & 2 y_1 + 3 y_2 + 4 y_3 + 5 y_4 \leq \zeta_1 - x_1 \\
                    & 6 y_1 + y_2 + 3 y_3 + 2 y_4 \leq \zeta_2 - x_2 \\
                    & y_i \in \{0,1\}
\end{align*}
$$
and $\zeta = (\zeta_1,\zeta_2)$ is uniformly distributed on
$$\Pi = \{(5,5), (5,6),\dots,(5,15),(6,5),\dots,(15,15)\}$$
giving 121 scenarios.

In [1]:
RELAX_LP = True

In [2]:
from __future__ import division
from cobra import Model
from dynamicme.optimize import Variable, Constraint
from six import iteritems
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel

import numpy as np
import pandas as pd

In [3]:
zetas = [(i,j) for i in np.arange(5,16) for j in np.arange(5,16)]
print('Scenarios: %s'%len(zetas))

Scenarios: 121


In [4]:
cx = -np.array([3/2, 4])
fy = -np.array([16, 19, 23, 28])
xl = [0, 0]
xu = [5, 5]
A  = [[2,3,4,5],
      [6,1,3,2]]

sub_dict = {}
for k,zeta in enumerate(zetas):
    mdl = Model('sub')    
    ys  = [Variable('y_%d'%j, lower_bound=0, upper_bound=1, objective_coefficient=fy[j]) for j in range(len(fy))]
    xs  = [Variable('x_%d'%j, lower_bound=xl[j], upper_bound=xu[j], objective_coefficient=cx[j]) for j in range(len(cx))]
    for x in xs:
        x.variable_kind = 'integer'
    for y in ys:
        y.variable_kind = 'integer'
    mdl.add_reactions(xs+ys)
    for i,ai in enumerate(A):
        cons = Constraint('cons_%d'%i)
        cons._constraint_sense = 'L'
        cons._bound = zeta[i]
        xs[i].add_metabolites({cons:1})
        for j,aij in enumerate(ai):
            ys[j].add_metabolites({cons:aij})
    sub_id = 'scen_%d'%k
    sub = LagrangeSubmodel(mdl, sub_id, first_stage_vars=xs)
    sub_dict[sub_id] = sub

In [5]:
master = LagrangeMaster(mdl, first_stage_vars=xs)
master._INF = 1e8
master.add_submodels(sub_dict)
master._z.LB = -master._INF
master._z.UB = master._INF
master.model.Params.Presolve = 0
master.model.update()
master.model.Params.NodefileStart = 3.
for sub in sub_dict.values():
    sub.model.Params.NodefileStart = 3.

In [ ]:
master.gaptol = 1e-3
master.penaltytol = 1e-4
master.delta_mult = 0

In [ ]:
master.verbosity=1
feasible_method = 'enumerate'
alt_method = 'pool'
max_alt = 10


if RELAX_LP:
    sol_master = master.solve_relaxed(feasible_method=None)
    df_phase1 = pd.DataFrame(master.log_rows)

sol_master = master.optimize(feasible_method=feasible_method, max_alt=max_alt, alt_method=alt_method) 
df_phase2 = pd.DataFrame(master.log_rows)

    Iter                    UB                    LB       gap relgap(%)   penalty                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Best   Feasible        Sub       Best                                   total    master       sub
       0      1e+08     1e+100      -8898      -8898     1e+08       100         0  0.048032  0.001148  0.044603
       7      -8095     1e+100      -8098      -8098      3.65    0.0451  9.65e-08  0.384240  0.011466  0.038301
    Iter                    UB                    LB       gap relgap(%)   penalty                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Best   Feasible        Sub       Best                                   total    master       sub
       0      -8501     1e+100      -8261      -8261     240.2     2.825       183  0.137004  0.

/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition.py:2145 UserWarning: Solution available but Master solver status=suboptimal (13).


      50      -8095     1e+100      -7936      -7936     158.8     1.961  4.62e-11  11.83940  0.317935  0.089411
      60      -8095     1e+100      -7936      -7936     158.8     1.961  3.94e-11  16.70945  0.435214  0.089844
      70      -8095     1e+100      -7936      -7926     168.5     2.082  3.22e-11  24.47460  0.642467  0.088242
      80      -8095     1e+100      -7936      -7926     168.5     2.082  1.84e-11  33.47116  1.069900  0.088378
      90      -8095     1e+100      -7936      -7926     168.5     2.082  2.11e-11  44.98542  1.218603  0.090847
     100      -8095     1e+100      -7936      -7926     168.5     2.082  4.89e-11  59.70765  1.488387  0.095020
     110      -8095     1e+100      -7936      -7926     168.5     2.082  2.56e-12  76.86680  1.681466  0.092315
     120      -8095     1e+100      -7936      -7926     168.5     2.082  7.45e-11  97.08871  2.531141  0.089874
     130      -8095     1e+100      -7936      -7926     168.5     2.082  4.42e-11  122.3122  2.

In [ ]:
sub.model.getObjective()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

xcol = 'iter'
df_phase1.loc[:,'phase'] = 1
df_phase1.loc[:,'feasUB'] = np.nan
df_phase2.loc[:,'phase'] = 2
dsplot = pd.concat([df_phase1, df_phase2])
dsplot = dsplot[[xcol,'feasUB','bestUB','bestLB','phase']]
dsplot = dsplot.melt(id_vars=[xcol,'phase'])

g = sns.FacetGrid(dsplot, hue='variable', size=3, aspect=3, row='phase', sharey=False)
g.map(plt.plot, xcol,'value')
g.add_legend()